# A Study of Transcription and Its Affects
---
**Note:** To get to my GitHub repository, go to this [link](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects).

I have broken this project into to major portions. The first portion calculates the hits (weak and strong transcription factor binding sights) for a specified chromosome and transcription factor. The second portion analyses these hits. Let's talk about these two portions more in depth.

## Part 1: Finding the Hits
---
To calculate the hits, I read in from a file called `input.txt` which is formatted as follows (please note that numbers are line numbers):

1. Name of fasta file
2. Name of chromosome
3. Name of tamo file
4. Name of the transcription factor you want to use
5. Weak threshold value
6. Strong threshold value
7. Size of each category of distances between sites
8. Maximum allowed plus/minus distance between a strong and weak site

I had originally sought out to do this project in [MatLab](https://www.mathworks.com), but with the fact that we're running our code with super big data, I have decided to use [Python](https://www.python.org) for this project. I read in the chromosome from a fasta file called [SGDv3.fasta](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects/blob/master/SGDv3.fasta) and the associated PSSM to the transcription factor from the tamo file called [yeast.tamo](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects/blob/master/yeast.tamo). Before we continue, let's first read in `input.txt`.

In [12]:
def readInInput(filename):
    # Open the file
    fileID = open(filename, 'r')
    # Initialize the list that will hold the input
    data = []
    # Read in the input file
    line = fileID.readline()
    while line:
        data.append(line[:len(line)-1])
        line = fileID.readline()
    # Convert into correct units
    data[4] = float(data[4])
    data[5] = float(data[5])
    data[6] = int(data[6])
    data[7] = int(data[7])
    # Return
    return data

# Test readInInput
Input = readInInput('input.txt')
print(Input)

['SGDv3.fasta', 'chr2', 'yeast.tamo', 'YAP5', 0.35, 0.7, 1, 15]


 Now that we've read in `input.txt`, let's read in, from SGDv3.fasta, all the chromosomes:

In [20]:
def getCHR(filename, chrName):
    # Open the fasta file
    fileID = open(filename, 'r')
    # Define the dictionary that will the hold the data from the chromosomes
    chrData = {}
    # Read in the fasta file
    line = fileID.readline()
    while line:
        if line[0] == '>':
            title = line[1:len(line)-1]
            chrData[title] = ''
        else:
            chrData[title] += line[:len(line)-1]
        line = fileID.readline()
    # Return the specified chromosome
    chrNum = int(chrName[2:]) - 1
    return chrData[chrNum]

After we read in all the different chromosomes, we also need to read in the corresponding PSSM to the given transcription factor.

In [11]:
def getPSSM(filename, TF):
    # Open the tamo file
    fileID = open(filename, 'r')
    # Read in the tamo file
    tamoData = []
    fileID = open(filename, 'r')
    line = fileID.readline()
    while line:
        tamoData.append(line[:len(line)-1])
        line = fileID.readline()
    # Define what we're looking for
    correctLine = 'Source:  ' + TF
    # Search for your correct line
    for i in range(123):
        # Get line 19 + 42*i
        line = tamoData[19 + 42*i - 1]
        # Compare
        if line == correctLine:
            position = i
    # Set the start line for the PSSM
    startLine = 2 + 42*position
    # Get the startLine's length to figure out the length of the PSSM
    firstLine = tamoData[startLine]
    lenOfPSSM = int((len(firstLine) - 3)/10) - 1
    # Create a list that will contain the PSSM
    PSSM = [[0 for i in range(lenOfPSSM)] for i in range(4)]
    print(PSSM)
    print(len(PSSM))
    # Get the all the lines of the PSSM
    firstLine = tamoData[startLine]
    secondLine = tamoData[startLine + 1]
    thirdLine = tamoData[startLine + 2]
    fourthLine = tamoData[startLine + 3]
    # Get the values and insert them into the PSSM
    for i in range(lenOfPSSM):
        print(i)
        value = firstLine[(i*10 + 6):(i*10 + 12)]
        PSSM[1][i] = float(value)
        print(value, float(value))
        value = secondLine[(i*10 + 6):(i*10 + 12)]
        PSSM[2][i] = float(value)
        print(value, float(value))
        value = thirdLine[(i*10 + 6):(i*10 + 12)]
        PSSM[3][i] = float(value)
        print(value, float(value))
        value = fourthLine[(i*10 + 6):(i*10 + 12)]
        PSSM[4][i] = float(value)
    
PSSM = getPSSM(Input[2], Input[3])

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
4
0
 1.232 1.232
-9.396 -9.396
-0.512 -0.512


IndexError: list index out of range

After we read in both the chromosomes and the corresponding PSSM for the given transcription factor, we need to write a function that caluclates the output of the PSSM when you give it a specific string.

In [8]:
def outputOfPSSM( PSSM, sequence, maximum, lenOfPSSM ):
    output = 1.0
    # Go through the PSSM and calculate the output for the given sequence
    for i in range(lenOfPSSM):
        if sequence[i] == 'A':
            output += PSSM[1][i]
        if sequence[i] == 'C':
            output += PSSM[2][i]
        if sequence[i] == 'T':
            output += PSSM[3][i]
        if sequence[i] == 'G':
            output += PSSM[4][i]
    output /= maximum
    # Return output
    return output

Now that we have all of these functions, let's use them to calculate the hits for the specified chromosome and transcription factor.

In [21]:
def calculateHits(filename):
    # Get the value in the file
    Input = readInInput(filename)
    # Get the chromosome
    chrData = getCHR(Input[0], Input[1])
    print(chrData)

calculateHits('input.txt')

ValueError: invalid literal for int() with base 10: 'r2'